In [59]:
import numpy as np
import pandas as pd
from sklearn.calibration import LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import SequentialFeatureSelector
import seaborn as sns
from sklearn.feature_selection import SequentialFeatureSelector
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
import statsmodels.api as sm


In [60]:
df1 = pd.read_csv("train.csv")
#df1.head()
df2 = pd.read_csv('test.csv')
#df2

In [61]:
# Identify and drop duplicate columns
duplicated_columns = df1.columns[df1.columns.duplicated()]
df_no_duplicates = df1.drop(columns=duplicated_columns)

In [62]:
# Identify and drop duplicate columns
duplicated_columns = df2.columns[df2.columns.duplicated()]
df_no_duplicates = df2.drop(columns=duplicated_columns)

In [63]:
df1 = df1.drop(columns=['sub_area'])
df2 = df2.drop(columns=['sub_area','row ID'])

In [ ]:
df_onehot = pd.get_dummies(df1)
df_onehot.dtypes
test_onehot = pd.get_dummies(df2)

In [65]:
df_onehot = pd.get_dummies(df1,drop_first=True)
test_onehot = pd.get_dummies(df2,drop_first=True)

In [ ]:
categorical_columns = df1.select_dtypes(include=['object']).columns

# Label encode categorical columns
label_encoder = LabelEncoder()
for col in categorical_columns:
   df1[col] = label_encoder.fit_transform(df1[col])
   df2[col] = label_encoder.transform(df2[col])

In [66]:
X = df_onehot.drop(columns=['price_doc',],axis=1)
y = df_onehot['price_doc']

In [ ]:
X = df1.drop(columns=['price_doc',],axis=1)
y = df1['price_doc']

In [ ]:
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)
df2_encoded = imputer.fit_transform(test_onehot)

In [67]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X_testscaled = scaler.fit_transform(test_onehot)

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
df2_encoded_scaled = scaler.fit_transform(df2_encoded)

In [68]:
#X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

variance_threshold = 0.01 # Set your desired threshold
selector = VarianceThreshold(threshold=variance_threshold)
X_train_high_variance = selector.fit_transform(X_scaled)
X_test_high_variance = selector.transform(X_testscaled)

In [ ]:
#covert x_scaled to dataframe
X_scaled = pd.DataFrame(X_scaled)
#convert x_testscaled to dataframe
X_testscaled = pd.DataFrame(X_testscaled)

In [ ]:
import statsmodels.api as sm
import pandas as pd

# Convert scaled_df1 to a DataFrame without specifying columns

# Add a constant term to the feature matrix
X_with_const = sm.add_constant(X_scaled)

# Fit a linear regression model
model = sm.OLS(y, X_with_const).fit()

# Get p-values for each feature
p_values = model.pvalues[1:]  # Exclude the constant term

# Set your desired threshold for p-value
threshold = 0.05

# Filter features based on p-value
selected_features = p_values[p_values < threshold].index

# Display selected features
print("Selected Features:")
print(selected_features)

# Select columns in the DataFrame
X_selected = X_scaled[selected_features]


In [ ]:
#keep those columns in X_testscaled that are in X_selected
X_testscaled = X_testscaled[X_selected.columns]

In [ ]:
X_selected.shape

In [ ]:
X_testscaled.shape

In [69]:
kBest = SelectKBest(k=20, score_func=f_regression)
X_train_kBest = kBest.fit_transform(X_train_high_variance, y)
X_test_kBest = kBest.transform(X_test_high_variance)

In [70]:
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_kBest)
X_test_pca = pca.transform(X_test_kBest)

In [72]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))  # Adding dropout with a rate of 0.2
model.add(Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))  # Adding L2 regularization
model.add(Dense(1, activation='sigmoid'))


Coefficients: [ 1.60937858e+06  4.53526120e+05 -1.10221598e+04  3.92892675e+05
 -5.53822162e+05 -6.26287712e+05  3.93226377e+05 -9.19674590e+05
 -3.26540844e+05 -6.64639372e+04 -2.66188010e+04  1.30103635e+04
  7.22925483e+03 -2.05020677e+04  2.30956406e+04  3.34749336e+03
 -2.33376168e+04  2.12535567e+04 -7.17926352e+03  2.91959380e+03
  7.30503151e+03 -3.98481178e+04  3.20317588e+03 -7.16956952e+04
 -1.64676190e+04  7.61660034e+03 -1.07973920e+05  2.12099980e+03
 -5.73519512e+04 -6.63921133e+03  2.36107813e+04 -6.24205087e+04
  3.46387632e+04  3.05445742e+04 -6.09886405e+04  1.72323150e+04
 -2.54696075e+03 -4.94898606e+02  9.40150213e+04 -5.18418633e+04
 -6.95941127e+04 -2.37317462e+04  4.73050913e+04 -3.04252008e+04
 -3.07130775e+03  1.09287820e+04  2.73366937e+04  2.03570068e+04
  2.17591981e+03  8.64827725e+03  4.73664242e+04 -5.30185315e+04
  3.83294341e+04  3.95847833e+04 -1.04922872e+05 -3.20152304e+03
  7.24496677e+04  2.04496305e+04  1.96088446e+04 -8.11350440e+03
  1.0211528

In [73]:
additional_data = pd.read_csv("test.csv")

# Create a DataFrame with 'row ID' and predicted values
predicted_df = pd.DataFrame({
    'row ID': additional_data['row ID'],
    'price_doc': md_probs  # Replace with the actual column name for your predictions
})

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions47.csv', index=False)